In [1]:
"""testing the multidim wav2vec model against the reference model"""
import torch
import numpy as np
from lightning.pytorch import seed_everything
import seisbench.data as sbd

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers import Wav2Vec2Config
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.multidim_wav2vec2 import MultiDimWav2Vec2ForPreTraining

data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms).unsqueeze(0)



2024-06-16 17:03:55,736 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [4]:
model_output = {}
# for model_name in MODEL_NAMES:
config = Wav2Vec2Config.from_pretrained(
  '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/pretrain/model_config_4xdownsample_sinkhorn.json'
)


# config.sinkhorn_quantization_iters = 1

for sinkhorn_quantization_iter in [0, 1, 3, 5]:
  print(f"Sinkhorn quantization iter: {sinkhorn_quantization_iter}")
  config.sinkhorn_quantization_iters = sinkhorn_quantization_iter
  model = MultiDimWav2Vec2ForPreTraining(config)

  # compute masked indices
  batch_size, num_channels, raw_sequence_length = input_values.shape
  sequence_length = model._get_feat_extract_output_lengths(
    raw_sequence_length).item()

  seed_everything(0)
  mask_time_indices = _compute_mask_indices(
      shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
  )
  sampled_negative_indices = _sample_negative_indices(
      features_shape=(batch_size, sequence_length),
      num_negatives=model.config.num_negatives,
      mask_time_indices=mask_time_indices,
  )
  mask_time_indices = torch.tensor(
    data=mask_time_indices, device=input_values.device, dtype=torch.long)
  sampled_negative_indices = torch.tensor(
    data=sampled_negative_indices, device=input_values.device,
    dtype=torch.long
  )

  with torch.no_grad():
    outputs = model(input_values, mask_time_indices=mask_time_indices,
                    sampled_negative_indices=sampled_negative_indices)


  print(outputs.codevector_perplexity)
  # model_output[f'{model_name}_{model_type}'] = outputs


Seed set to 0


codevector_probs.shape torch.Size([2998, 320])
codevector_probs.sum(0) tensor([11., 11.,  4.,  2.,  6.,  3.,  8.,  9., 14., 13.,  1.,  7.,  6.,  5.,
         8.,  9.,  5.,  4.,  4.,  7.,  7., 15.,  4., 27.,  7.,  6., 17., 10.,
        16.,  4.,  8.,  6., 17.,  6.,  8.,  2.,  3.,  2.,  4., 11., 13., 16.,
         4.,  7.,  7.,  7.,  5., 10., 15., 12.,  3., 10., 10.,  4.,  9., 11.,
        17.,  4., 15., 22., 41.,  5.,  9.,  7.,  9.,  4.,  4.,  1., 35.,  8.,
         6.,  2., 16.,  6.,  9., 12., 11.,  4.,  5., 11.,  9.,  8.,  7., 13.,
        12.,  6.,  9.,  4.,  9.,  2.,  4., 12., 12.,  4.,  5.,  9.,  9.,  2.,
         4.,  5., 21.,  2., 33.,  2.,  1.,  9.,  8.,  6.,  9.,  8.,  4.,  7.,
         6., 12.,  4.,  5., 11., 25.,  2.,  3., 20.,  8.,  5., 14.,  5., 12.,
         7.,  4.,  3.,  5., 19.,  8., 43., 12.,  2., 18., 25.,  5., 14.,  6.,
         3.,  4.,  6.,  5.,  4.,  7., 11., 29., 12., 11.,  5.,  5.,  6.,  6.,
         4., 10., 21., 35., 20., 16., 12., 10.,  6.,  7.,  5., 19., 10.

In [ ]:
# Seed set to 0
# codevector_probs.shape torch.Size([2998, 320])
# codevector_probs.sum(0) tensor([10., 10.,  7.,  8., 11.,  9.,  9., 11., 10., 11., 12., 10.,  9.,  8.,
#          8.,  5.,  8.,  9., 10., 11., 10.,  9.,  9.,  7.,  8.,  9.,  7.,  8.,
#         10., 10.,  9., 10., 10.,  8.,  8.,  8.,  9., 10., 11., 11., 11.,  8.,
#         11.,  9.,  9., 10., 10.,  9., 10.,  7., 10.,  9.,  9., 11., 10.,  8.,
#         11.,  9., 10.,  9., 12., 11.,  9.,  8.,  9., 10.,  7.,  8.,  8., 10.,
#          9.,  9., 11., 10., 10.,  8., 11.,  8.,  6.,  9., 11.,  9., 10., 11.,
#         11.,  8.,  8., 12.,  9., 11.,  9., 12., 11., 12.,  9.,  8., 10., 10.,
#          9.,  8.,  8.,  7., 10.,  8.,  9.,  8.,  9.,  8.,  8., 13., 11.,  8.,
#          8.,  9.,  9.,  8.,  9.,  9., 11., 10., 11.,  8., 11.,  9., 10., 10.,
#          8., 10.,  6., 10., 10.,  6.,  9., 10.,  9.,  9.,  9.,  8., 13.,  9.,
#         10.,  9., 12., 10.,  7.,  9., 10.,  8.,  8., 11., 10.,  9.,  9.,  8.,
#         10., 10.,  7.,  6.,  9.,  5.,  8., 10., 10.,  8., 11.,  9., 10., 11.,
#          7.,  8.,  9.,  9.,  6.,  8., 10., 10.,  9., 12.,  9.,  9., 10.,  7.,
#          8., 10.,  8.,  9., 10.,  9., 12., 10., 12., 11.,  9., 10.,  9.,  9.,
#         10., 12., 10., 10.,  9.,  9., 14., 10.,  9.,  8., 11., 10., 11.,  9.,
#          9.,  8.,  9., 11., 11.,  8.,  9., 10.,  7., 10., 11., 10., 12., 11.,
#         10.,  8.,  9., 10., 11.,  7.,  8.,  7., 11.,  9., 11.,  9., 10.,  9.,
#          8.,  8.,  8., 12.,  9., 12., 12., 11., 10.,  8., 10., 10., 10.,  8.,
#          9.,  8., 12., 12., 12.,  9.,  9.,  5.,  8.,  9., 11.,  9., 10., 10.,
#          9., 11., 11.,  9.,  9.,  8., 10.,  9., 10.,  9.,  9.,  6.,  9.,  7.,
#         10.,  8., 10.,  8.,  9.,  9., 11., 10., 11., 10.,  9.,  8.,  9., 10.,
#         10., 10., 10.,  9.,  8., 14., 13.,  8., 10.,  9.,  8., 11.,  9.,  8.,
#         10.,  9.,  8., 10.,  6., 10., 12.,  9., 10.,  8.,  8., 11.])
# codevector_probs.sum(1) tensor([1., 1., 1.,  ..., 1., 1., 1.])
# tensor(168.0099)